In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import gmaps
import gmaps.geojson_geometries
import os
import json
import requests
import pprint as pp

import csv



# Import API key
from config import g_key

ModuleNotFoundError: No module named 'config'

In [2]:
# File to Load
NYC1_data_to_load = "resources/NYC_Listings1.csv"
NYC2_data_to_load = "resources/NYC_Listings2.csv"
NYC3_data_to_load = "resources/NYC_Listings3.csv"

# Read School and Student Data File and store into Pandas Data Frames
NYC1_data = pd.read_csv(NYC1_data_to_load)
NYC2_data = pd.read_csv(NYC2_data_to_load)
NYC3_data = pd.read_csv(NYC3_data_to_load)

/Users/mikeyesteban/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mikeyesteban/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mikeyesteban/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Create a full NYC DF
NYC_data = pd.concat([NYC1_data, NYC2_data, NYC3_data], ignore_index=True)
NYC_data.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,2.019080e+13,8/7/19,Skylit Midtown Castle,"Find your romantic getaway to this beautiful, ...","- Spacious (500+ft²), immaculate and nicely fu...","Find your romantic getaway to this beautiful, ...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,2.0,1.0,0.0,1.0,0.39
1,3647,https://www.airbnb.com/rooms/3647,2.019080e+13,8/6/19,THE VILLAGE OF HARLEM....NEW YORK !,NaN,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,none,NaN,...,f,f,strict_14_with_grace_period,t,t,1.0,0.0,1.0,0.0,NaN
2,3831,https://www.airbnb.com/rooms/3831,2.019080e+13,8/6/19,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1.0,1.0,0.0,0.0,4.64
3,5022,https://www.airbnb.com/rooms/5022,2.019080e+13,8/6/19,Entire Apt: Spacious Studio/Loft by central park,NaN,Loft apartment with high ceiling and wood floo...,Loft apartment with high ceiling and wood floo...,none,NaN,...,f,f,strict_14_with_grace_period,t,t,1.0,1.0,0.0,0.0,0.10
4,5099,https://www.airbnb.com/rooms/5099,2.019080e+13,8/6/19,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment is true New York ...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment is true New York ...,none,My neighborhood in Midtown East is called Murr...,...,f,f,strict_14_with_grace_period,t,t,1.0,1.0,0.0,0.0,0.60


In [4]:
# Review columns
column_names = list(NYC_data.columns)
column_names[:5]

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name']

In [5]:
# Select Columns for clean DataFrame
Clean_NYC = NYC_data[['id','name','neighbourhood',
                      'latitude','longitude','property_type',
                      'room_type','accommodates','amenities',
                      'price','availability_365','number_of_reviews',
                     'number_of_reviews_ltm','review_scores_rating',
                      'review_scores_location','reviews_per_month']]
# Drop null values
Clean_NYC = Clean_NYC.dropna()

# Change price value to float
Clean_NYC['price'] = Clean_NYC['price'].map(lambda x: x.lstrip('$'))
#Clean_NYC['price'] = Clean_NYC['price'].map(lambda x: x.lstrip(','))
Clean_NYC['price'] = Clean_NYC['price'].map(lambda x: x.replace(',',''))
Clean_NYC['price'] = Clean_NYC['price'].astype('float')

# Preview DataFrame
Clean_NYC

,id,name,neighbourhood,latitude,longitude,property_type,room_type,accommodates,amenities,price,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,reviews_per_month
0,2595,Skylit Midtown Castle,Midtown,40.75362,-73.98377,Apartment,Entire home/apt,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",225.0,288.0,46.0,12.0,95.0,10.0,0.39
2,3831,Cozy Entire Floor of Brownstone,Brooklyn,40.68514,-73.95976,Guest suite,Entire home/apt,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",89.0,212.0,274.0,70.0,90.0,10.0,4.64
3,5022,Entire Apt: Spacious Studio/Loft by central park,East Harlem,40.79851,-73.94399,Apartment,Entire home/apt,1.0,"{Internet,Wifi,""Air conditioning"",Kitchen,Elev...",80.0,0.0,9.0,2.0,93.0,9.0,0.10
4,5099,Large Cozy 1 BR Apartment In Midtown East,Midtown East,40.74767,-73.97500,Apartment,Entire home/apt,2.0,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Buzzer/w...",200.0,127.0,75.0,10.0,89.0,9.0,0.60
5,5121,BlissArtsSpace!,Brooklyn,40.68688,-73.95596,Apartment,Private room,2.0,"{Wifi,""Air conditioning"",Kitchen,""Pets live on...",60.0,0.0,49.0,0.0,90.0,9.0,0.39
6,5178,Large Furnished Room Near B'way,Manhattan,40.76489,-73.98493,Apartment,Private room,2.0,"{TV,Wifi,""Air conditioning"",""Paid parking off ...",79.0,239.0,434.0,44.0,84.0,10.0,3.48
7,5203,Cozy Clean Guest Room - Family Apt,Upper West Side,40.80178,-73.96723,Apartment,Private room,1.0,"{Internet,Wifi,""Air conditioning"",""Paid parkin...",79.0,0.0,118.0,0.0,98.0,10.0,0.98
8,5222,Best Hideaway,Manhattan,40.72764,-73.97949,Apartment,Entire home/apt,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",116.0,322.0,94.0,0.0,95.0,10.0,0.74
9,5238,Cute & Cozy Lower East Side 1 bdrm,Lower East Side,40.71344,-73.99037,Apartment,Entire home/apt,3.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",150.0,0.0,161.0,13.0,94.0,9.0,1.33
10,5295,Beautiful 1br on Upper West Side,Manhattan,40.80316,-73.96545,Apartment,Entire home/apt,2.0,"{Internet,Wifi,""Air conditioning"",Kitchen,Door...",135.0,1.0,53.0,14.0,93.0,9.0,0.42


In [112]:
# GroupBY Neighborhood
neighborhoods_ratings = Clean_NYC.groupby('neighbourhood')['review_scores_location'].mean()
neighborhoods_ratings.sort_values(ascending=False, inplace=True)
neighborhoods_ratings.head()
# USE GOOGLE PLACES API TO CREATE A MAP THATS SPLIT

neighbourhood
Times Square/Theatre District    10.0
Grant City                       10.0
City Island                      10.0
Oakwood                          10.0
Woodlawn                         10.0
Name: review_scores_location, dtype: float64

In [734]:
neighborhoods_list = list(neighborhoods_ratings.index)
neighborhoods_list[:10]
len(neighborhoods_list)

190

In [735]:
# Create a Dictionary with neighbourhood DataFrames
d = {}
for city in neighborhoods_list:
    checker = (Clean_NYC['neighbourhood'] == city)
    city_df = Clean_NYC.where(checker)
    city_df = city_df.dropna()
    d[city] = city_df


In [736]:
final_neighborhood_list = list(d.keys())
final_neighborhood_list.sort()
final_neighborhood_list

['Allerton',
 'Alphabet City',
 'Annadale',
 'Arrochar',
 'Astoria',
 'Bath Beach',
 'Battery Park City',
 'Bay Ridge',
 'Baychester',
 'Bayside',
 'Bedford Park',
 'Bedford-Stuyvesant',
 'Belmont',
 'Bensonhurst',
 'Bergen Beach',
 'Boerum Hill',
 'Borough Park',
 'Brighton Beach',
 'Bronxdale',
 'Brooklyn',
 'Brooklyn Heights',
 'Brooklyn Navy Yard',
 'Brownsville',
 'Bushwick',
 'Canarsie',
 'Carroll Gardens',
 'Castle Hill ',
 'Castleton Corners',
 'Chelsea',
 'Chinatown',
 'City Island',
 'Civic Center',
 'Claremont',
 'Clinton Hill',
 'Cobble Hill',
 'College Point',
 'Columbia Street Waterfront',
 'Concord',
 'Concourse',
 'Concourse Village',
 'Coney Island',
 'Corona',
 'Country Club',
 'Crotona',
 'Crown Heights',
 'DUMBO',
 'Ditmars / Steinway',
 'Dongan Hills',
 'Downtown Brooklyn',
 'Dyker Heights',
 'East Elmhurst',
 'East Flatbush',
 'East Harlem',
 'East New York',
 'East Village',
 'Eastchester',
 'Edenwald',
 'Elm Park',
 'Elmhurst',
 'Eltingville',
 'Financial Distri

### CHANGING NAMES AND UPDATING DICTIONARY

In [115]:
# Function to Change Names
def change_name(self,key,new_key):
    value = self[key]
    self.pop(key)
    self[new_key] = value

In [118]:
# Functino to Update Dictionary
def title_name_change(self,old_key, new_key):
    change_name(self,old_key,new_key)
    dataframe = self[new_key]
    dataframe['new neighborhood'] = new_key

In [733]:
title_name_change(d,"Williamsbridge","Williamsbridge-Olinville")
d["Williamsbridge-Olinville"].head()



,id,name,neighbourhood,latitude,longitude,property_type,room_type,accommodates,amenities,price,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,reviews_per_month,new neighborhood
2863,1693028.0,Private Room with adjoining bathroom,Williamsbridge,40.88296,-73.86264,House,Private room,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",50.0,340.0,20.0,5.0,90.0,9.0,0.84,Williamsbridge-Olinville
6532,4795390.0,Cozy Private Room,Williamsbridge,40.88016,-73.84923,Townhouse,Private room,1.0,"{Internet,Wifi,Kitchen,""Free street parking"",H...",43.0,365.0,15.0,3.0,92.0,9.0,0.29,Williamsbridge-Olinville
10566,8275331.0,Affordable and Spacious room,Williamsbridge,40.88301,-73.85452,House,Private room,1.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",65.0,133.0,19.0,2.0,87.0,9.0,0.43,Williamsbridge-Olinville
12254,9597258.0,Cozy and affordable room.,Williamsbridge,40.88165,-73.85625,House,Private room,1.0,"{TV,""Cable TV"",Wifi,Kitchen,""Free street parki...",34.0,284.0,18.0,3.0,92.0,9.0,0.41,Williamsbridge-Olinville
29436,23100862.0,Heaven On Earth,Williamsbridge,40.88049,-73.86130,House,Private room,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,Heating,""S...",41.0,0.0,1.0,0.0,100.0,10.0,0.06,Williamsbridge-Olinville


In [724]:
# ADDING LISTS TOGETHER
d["Woodlawn-Wakefield"] = d["Woodlawn-Wakefield"].append(d["Woodlawn"], ignore_index=False)



In [725]:
# adding new neighborhood name
d["Woodlawn-Wakefield"]['new neighborhood'] = "Woodlawn-Wakefield"



In [729]:
#d['SoHo-TriBeCa-Civic Center-Little Italy']
#list(d.keys())
#d['DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill']
(d["Washington Heights North"])

KeyError: 'Washington Heights North'

In [699]:
final_neighborhood_list


['Allerton-Pelham Gardens',
 "Annadale-Huguenot-Prince's Bay-Eltingville",
 'Astoria',
 'Bath Beach',
 'Battery Park City-Lower Manhattan',
 'Bay Ridge',
 'Bayside-Bayside Hills',
 'Bedford',
 'Bedford Park-Fordham North',
 'Belmont',
 'Bensonhurst West',
 'Borough Park',
 'Breezy Point-Belle Harbor-Rockaway Park-Broad Channel',
 'Brighton Beach',
 'Bronxdale',
 'Brooklyn',
 'Brooklyn Heights-Cobble Hill',
 'Brownsville',
 'Bushwick South',
 'Canarsie',
 'Carroll Gardens-Columbia Street-Red Hook',
 'Castleton Corners',
 'Chinatown',
 'Claremont-Bathgate',
 'Clinton',
 'Clinton Hill',
 'College Point',
 'Corona',
 'Crotona Park East',
 'Crown Heights North',
 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill',
 'Dyker Heights',
 'East Concourse-Concourse Village',
 'East Elmhurst',
 'East Flatbush-Farragut',
 'East Harlem South',
 'East New York',
 'East Village',
 'Eastchester-Edenwald-Baychester',
 'Elmhurst',
 'Flatbush',
 'Flatlands',
 'Flushing',
 'Forest Hills',
 'Fort Greene',
 '

In [726]:
d.pop("Woodlawn")

,id,name,neighbourhood,latitude,longitude,property_type,room_type,accommodates,amenities,price,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,reviews_per_month
19831,16149186.0,Cozy private room in the Bronx!,Woodlawn,40.89743,-73.86983,House,Private room,1.0,"{TV,Wifi,""Air conditioning"",Kitchen,Heating,""S...",68.0,65.0,1.0,0.0,100.0,10.0,0.07


In [150]:
new_neighborhood_json_list = []

with open('resources/nyc-neighborhoods.geo.json') as f:
    geometry = json.load(f)
    
#     for key,value in geometry.items():
#         print(geometry['features'][0])

    for row in geometry['features']:
        city = row['properties']['name']
        if city not in new_neighborhood_json_list:
            new_neighborhood_json_list.append(city)

len(new_neighborhood_json_list)

188

In [732]:
for row in new_neighborhood_json_list:
    if "Williamsbridge" in row:
        print(row)

Williamsbridge-Olinville


In [700]:
# DATA MUNGING TO FIX CITY MAP
final_neighborhood_list
new_neighborhood_json_list
new_missing_cities = []
new_extra_json_cities = []


for city in final_neighborhood_list:
    if city not in new_neighborhood_json_list:
        new_missing_cities.append(city)
        
for city in new_neighborhood_json_list:
    if city not in final_neighborhood_list:
        new_extra_json_cities.append(city)

print('MISSING CITIES TO ADD', len(new_missing_cities))
print('EXTRA CITIES IN GRAPH', len(new_extra_json_cities))
new_missing_cities



#ALPHABETIZE Lists
new_neighborhood_json_list.sort()
new_missing_cities.sort()
len(new_neighborhood_json_list)


checker_df = pd.DataFrame({'missing cities': pd.Series(new_missing_cities), 
              'json list': pd.Series(new_neighborhood_json_list),
             'extra json to use': pd.Series(new_extra_json_cities)})
checker_df

#'Williamsburg' in new_neighborhood_json_list

MISSING CITIES TO ADD 17
EXTRA CITIES IN GRAPH 70


,missing cities,json list,extra json to use
0,Brooklyn,Allerton-Pelham Gardens,Arden Heights
1,Castleton Corners,Annadale-Huguenot-Prince's Bay-Eltingville,Auburndale
2,Manhattan,Arden Heights,Baisley Park
3,Queens,Astoria,Bellerose
4,South Street Seaport,Auburndale,Bensonhurst East
5,St. George,Baisley Park,Briarwood-Jamaica Hills
6,Staten Island,Bath Beach,Bushwick North
7,The Bronx,Battery Park City-Lower Manhattan,Cambria Heights
8,Tottenville,Bay Ridge,Central Harlem North-Polo Grounds
9,Tremont,Bayside-Bayside Hills,Central Harlem South


# THE NEW NEIGHBORHOOD COLUMN IS THE SAME AS JSON NAMES

In [ ]:
# # Configure gmaps with API key
# gmaps.configure(api_key=g_key)
# # Create a Gmaps figure
# fig = gmaps.figure()

# locations = Clean_NYC[['latitude','longitude']]
# weights = Clean_NYC['review_scores_location']
# # Create the Heat Layer Object 
# heat_layer = gmaps.heatmap_layer(locations=locations, weights=weights, dissipating=True)
# heat_layer.point_radius = 4
# heat_layer.gradient = [
#     'gray',
#     'black',
#     'blue'
# ]
# # Add Heat Layer to Figure
# fig.add_layer(heat_layer)

# fig

In [ ]:
# new_york_coordinates = (40.75, -74.00)
# #gmaps.figure(center=new_york_coordinates, zoom_level=12)

# countries_geojson = gmaps.geojson_geometries.load_geometry('us-counties')

# fig = gmaps.figure(center=new_york_coordinates, zoom_level=12)
# gini_layer = gmaps.geojson_layer(countries_geojson)
# fig.add_layer(gini_layer)
# #fig


In [ ]:
unfinished_df = pd.concat(d)
unfinished_df

In [ ]:
unfinished_df.to_csv(r'checker.csv', index=True, header=True)

In [3]:
data = pd.read_csv('checker.csv')
data.head()

,Unnamed: 0,Unnamed: 1,accommodates,amenities,availability_365,id,latitude,longitude,name,neighbourhood,new neighborhood,number_of_reviews,number_of_reviews_ltm,price,property_type,review_scores_location,review_scores_rating,reviews_per_month,room_type
0,Allerton-Pelham Gardens,1550,2.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",279.0,715270.0,40.85956,-73.87067,2 Beds/Queen & Full Beautiful Room 40 minsT.Sq...,Allerton,Allerton-Pelham Gardens,170.0,20.0,39.0,Apartment,9.0,90.0,2.05,Private room
1,Allerton-Pelham Gardens,1640,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",234.0,755528.0,40.85840,-73.86969,PRIVATE BATH/TONS OF SUNLIGHT/SAFE,Allerton,Allerton-Pelham Gardens,192.0,46.0,49.0,Apartment,9.0,90.0,2.33,Entire home/apt
2,Allerton-Pelham Gardens,1678,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",253.0,773041.0,40.85914,-73.86979,Nice beautiful room In the Bronx,Allerton,Allerton-Pelham Gardens,191.0,61.0,38.0,Apartment,10.0,88.0,2.36,Private room
3,Allerton-Pelham Gardens,22681,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,Heating,""F...",0.0,18702075.0,40.87309,-73.86326,Clean and large bedroom in a private house,Allerton,Allerton-Pelham Gardens,5.0,0.0,60.0,Apartment,9.0,87.0,0.19,Private room
4,Allerton-Pelham Gardens,25758,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Free stre...",235.0,20904409.0,40.86144,-73.86427,Luxury Tiny house • Ohka,Allerton,Allerton-Pelham Gardens,101.0,55.0,79.0,Guest suite,10.0,97.0,4.53,Entire home/apt


In [8]:
data = data.rename(columns={'Unnamed: 0':'city'})
data

,city,Unnamed: 1,accommodates,amenities,availability_365,id,latitude,longitude,name,neighbourhood,new neighborhood,number_of_reviews,number_of_reviews_ltm,price,property_type,review_scores_location,review_scores_rating,reviews_per_month,room_type
0,Allerton-Pelham Gardens,1550,2.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",279.0,715270.0,40.85956,-73.87067,2 Beds/Queen & Full Beautiful Room 40 minsT.Sq...,Allerton,Allerton-Pelham Gardens,170.0,20.0,39.0,Apartment,9.0,90.0,2.05,Private room
1,Allerton-Pelham Gardens,1640,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",234.0,755528.0,40.85840,-73.86969,PRIVATE BATH/TONS OF SUNLIGHT/SAFE,Allerton,Allerton-Pelham Gardens,192.0,46.0,49.0,Apartment,9.0,90.0,2.33,Entire home/apt
2,Allerton-Pelham Gardens,1678,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",253.0,773041.0,40.85914,-73.86979,Nice beautiful room In the Bronx,Allerton,Allerton-Pelham Gardens,191.0,61.0,38.0,Apartment,10.0,88.0,2.36,Private room
3,Allerton-Pelham Gardens,22681,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,Heating,""F...",0.0,18702075.0,40.87309,-73.86326,Clean and large bedroom in a private house,Allerton,Allerton-Pelham Gardens,5.0,0.0,60.0,Apartment,9.0,87.0,0.19,Private room
4,Allerton-Pelham Gardens,25758,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Free stre...",235.0,20904409.0,40.86144,-73.86427,Luxury Tiny house • Ohka,Allerton,Allerton-Pelham Gardens,101.0,55.0,79.0,Guest suite,10.0,97.0,4.53,Entire home/apt
5,Allerton-Pelham Gardens,27384,4.0,"{TV,""Air conditioning"",""Smoking allowed"",""Pets...",0.0,21830694.0,40.85973,-73.86699,A Piece of mind in the city,Allerton,Allerton-Pelham Gardens,7.0,0.0,70.0,Apartment,9.0,80.0,0.34,Entire home/apt
6,Allerton-Pelham Gardens,32918,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,Breakfast,...",36.0,26559892.0,40.86014,-73.86301,"== Modern, A/C, Easy Check-in / Book Instantly ==",Allerton,Allerton-Pelham Gardens,30.0,29.0,142.0,House,10.0,99.0,2.36,Private room
7,Allerton-Pelham Gardens,36550,4.0,"{TV,Wifi,Kitchen,""Pets allowed"",""Free street p...",187.0,29674940.0,40.86923,-73.86275,Sunlit Backyard in NYC + Walk to Zoo and Gardens!,Allerton,Allerton-Pelham Gardens,1.0,1.0,90.0,Townhouse,10.0,100.0,0.14,Entire home/apt
8,Allerton-Pelham Gardens,36723,4.0,"{Wifi,""Air conditioning"",Kitchen,""Free street ...",348.0,29800915.0,40.86116,-73.86248,Entire floor (private entrance) w/ 1 BR in NYC,Allerton,Allerton-Pelham Gardens,27.0,27.0,55.0,House,10.0,87.0,3.09,Entire home/apt
9,Allerton-Pelham Gardens,37458,4.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",364.0,30225205.0,40.86757,-73.86490,Relax De Dios,Allerton,Allerton-Pelham Gardens,1.0,1.0,60.0,Dome house,10.0,100.0,0.14,Private room


In [11]:
final_city_list = []
big_list = list(data['city'])
for row in big_list:
    if row not in final_city_list:
        final_city_list.append(row)

In [12]:
final_city_list

['Allerton-Pelham Gardens',
 "Annadale-Huguenot-Prince's Bay-Eltingville",
 'Astoria',
 'Bath Beach',
 'Battery Park City-Lower Manhattan',
 'Bay Ridge',
 'Bayside-Bayside Hills',
 'Bedford',
 'Bedford Park-Fordham North',
 'Belmont',
 'Bensonhurst West',
 'Borough Park',
 'Brighton Beach',
 'Bronxdale',
 'Brooklyn',
 'Brooklyn Heights-Cobble Hill',
 'Brownsville',
 'Bushwick South',
 'Canarsie',
 'Carroll Gardens-Columbia Street-Red Hook',
 'Castleton Corners',
 'Chinatown',
 'Claremont-Bathgate',
 'Clinton',
 'Clinton Hill',
 'College Point',
 'Corona',
 'Crotona Park East',
 'Crown Heights North',
 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill',
 'Dyker Heights',
 'East Concourse-Concourse Village',
 'East Elmhurst',
 'East Flatbush-Farragut',
 'East Harlem South',
 'East New York',
 'East Village',
 'Eastchester-Edenwald-Baychester',
 'Elmhurst',
 'Flatbush',
 'Flatlands',
 'Flushing',
 'Forest Hills',
 'Fort Greene',
 'Fresh Meadows-Utopia',
 'Georgetown-Marine Park-Bergen Bea

In [14]:
# Create a Dictionary with neighbourhood DataFrames
final_dict = {}
for city in final_city_list:
    checker = (data['city'] == city)
    city_df = data.where(checker)
    city_df = city_df.dropna()
    final_dict[city] = city_df

In [17]:
final_dict['Allerton-Pelham Gardens']

,city,Unnamed: 1,accommodates,amenities,availability_365,id,latitude,longitude,name,neighbourhood,new neighborhood,number_of_reviews,number_of_reviews_ltm,price,property_type,review_scores_location,review_scores_rating,reviews_per_month,room_type
0,Allerton-Pelham Gardens,1550.0,2.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",279.0,715270.0,40.85956,-73.87067,2 Beds/Queen & Full Beautiful Room 40 minsT.Sq...,Allerton,Allerton-Pelham Gardens,170.0,20.0,39.0,Apartment,9.0,90.0,2.05,Private room
1,Allerton-Pelham Gardens,1640.0,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",234.0,755528.0,40.85840,-73.86969,PRIVATE BATH/TONS OF SUNLIGHT/SAFE,Allerton,Allerton-Pelham Gardens,192.0,46.0,49.0,Apartment,9.0,90.0,2.33,Entire home/apt
2,Allerton-Pelham Gardens,1678.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",253.0,773041.0,40.85914,-73.86979,Nice beautiful room In the Bronx,Allerton,Allerton-Pelham Gardens,191.0,61.0,38.0,Apartment,10.0,88.0,2.36,Private room
3,Allerton-Pelham Gardens,22681.0,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,Heating,""F...",0.0,18702075.0,40.87309,-73.86326,Clean and large bedroom in a private house,Allerton,Allerton-Pelham Gardens,5.0,0.0,60.0,Apartment,9.0,87.0,0.19,Private room
4,Allerton-Pelham Gardens,25758.0,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Free stre...",235.0,20904409.0,40.86144,-73.86427,Luxury Tiny house • Ohka,Allerton,Allerton-Pelham Gardens,101.0,55.0,79.0,Guest suite,10.0,97.0,4.53,Entire home/apt
5,Allerton-Pelham Gardens,27384.0,4.0,"{TV,""Air conditioning"",""Smoking allowed"",""Pets...",0.0,21830694.0,40.85973,-73.86699,A Piece of mind in the city,Allerton,Allerton-Pelham Gardens,7.0,0.0,70.0,Apartment,9.0,80.0,0.34,Entire home/apt
6,Allerton-Pelham Gardens,32918.0,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,Breakfast,...",36.0,26559892.0,40.86014,-73.86301,"== Modern, A/C, Easy Check-in / Book Instantly ==",Allerton,Allerton-Pelham Gardens,30.0,29.0,142.0,House,10.0,99.0,2.36,Private room
7,Allerton-Pelham Gardens,36550.0,4.0,"{TV,Wifi,Kitchen,""Pets allowed"",""Free street p...",187.0,29674940.0,40.86923,-73.86275,Sunlit Backyard in NYC + Walk to Zoo and Gardens!,Allerton,Allerton-Pelham Gardens,1.0,1.0,90.0,Townhouse,10.0,100.0,0.14,Entire home/apt
8,Allerton-Pelham Gardens,36723.0,4.0,"{Wifi,""Air conditioning"",Kitchen,""Free street ...",348.0,29800915.0,40.86116,-73.86248,Entire floor (private entrance) w/ 1 BR in NYC,Allerton,Allerton-Pelham Gardens,27.0,27.0,55.0,House,10.0,87.0,3.09,Entire home/apt
9,Allerton-Pelham Gardens,37458.0,4.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",364.0,30225205.0,40.86757,-73.86490,Relax De Dios,Allerton,Allerton-Pelham Gardens,1.0,1.0,60.0,Dome house,10.0,100.0,0.14,Private room
